In [21]:
# importando bibliotecas conexão PDGT
import json
import argparse
import subprocess
import boto3
import time
from pyathena import connect
import pandas.io.sql as sqlio
import sys
from ydata_profiling import ProfileReport
from botocore import UNSIGNED
from botocore.config import Config
import boto3.session
from botocore import exceptions
from scipy.stats import chi2_contingency

In [22]:
#importando bibliotecas padrão
import pandas as pd
from datetime import datetime
from validate_docbr import CPF
import matplotlib.pyplot as plt

In [23]:
class CustomException(Exception):
    pass

json_manifest_dbt = "target/manifest.json"
athena_bucket = "todos-athena-us-east-1"
athena_tmp_folder = f"s3://{athena_bucket}/"

def execute_athena_sql(query):
    client = boto3.client('athena', region_name='us-east-1')
    queryStart = client.start_query_execution(
    QueryString = query,
    ResultConfiguration = { 'OutputLocation': athena_tmp_folder})
    queryExecution = client.get_query_execution(QueryExecutionId=queryStart['QueryExecutionId'])
    while queryExecution['QueryExecution']['Status']['State'] in ('RUNNING', 'QUEUED'):
        time.sleep(5)
        queryExecution = client.get_query_execution(QueryExecutionId=queryStart['QueryExecutionId'])


def execute_athena_query(query):
    cursor = connect(s3_staging_dir=athena_tmp_folder,
                    region_name="us-east-1").cursor()
    cursor.execute(query)
    colls=','.join(str(f"{e[0]}") for e in cursor.description)
    results=pd.DataFrame(list(cursor), columns=colls.split(","))
    return results

In [24]:
df_orc = pd.read_csv("/home/lucasmateus/dbt-projects/projetos_lucas/crm/modelo_campanha/extracoes_hub/outubro_rosa_cdt.csv")
df_orp = pd.read_csv("/home/lucasmateus/dbt-projects/projetos_lucas/crm/modelo_campanha/extracoes_hub/outubro_rosa_particular.csv")
df_nac = pd.read_csv("/home/lucasmateus/dbt-projects/projetos_lucas/crm/modelo_campanha/extracoes_hub/novembro_azul_cdt.csv")
df_nap = pd.read_csv("/home/lucasmateus/dbt-projects/projetos_lucas/crm/modelo_campanha/extracoes_hub/novembro_azul_particular.csv")
df_tlm = pd.read_csv("/home/lucasmateus/dbt-projects/projetos_lucas/crm/modelo_campanha/extracoes_hub/telemed_dezembro.csv")

In [25]:
df_orc

,Destinatário,ID do hub,ID da campanha de e-mail,Subtipo,Assunto,Enviado às (seu fuso horário),Motivo do não envio,Mensagem não enviada,Motivo da rejeição,Mensagem rejeitada,Enviados,Entregue,Suprimido,Recusado,Rejeição,Relatório de spam,Aberto,Clicado,Assinatura cancelada
0,ruizyara123@gmail.com,5338832,278742331,NaN,"Outubro Rosa, consulte condições e agende já!",2023-10-17 17:15:22,NaN,NaN,NaN,NaN,True,True,False,False,False,False,False,False,False
1,edileneaparecidadelima97@gmail.com,5338832,278742331,NaN,"Outubro Rosa, consulte condições e agende já!",2023-10-17 17:15:03,NaN,NaN,NaN,NaN,True,True,False,False,False,False,False,False,False
2,mariainestedesco2@gmail.com,5338832,278742331,NaN,"Outubro Rosa, consulte condições e agende já!",2023-10-17 17:15:08,NaN,NaN,NaN,NaN,True,True,False,False,False,False,False,False,False
3,raizaamorim6@gmail.com,5338832,278742331,NaN,"Outubro Rosa, consulte condições e agende já!",2023-10-18 07:01:22,NaN,NaN,NaN,NaN,True,True,False,False,False,False,False,False,False
4,marina_rolim@yahoo.com,5338832,278742331,NaN,"Outubro Rosa, consulte condições e agende já!",2023-10-17 17:15:21,NaN,NaN,NaN,NaN,True,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121801,cuidadoris75@gmail.com,5338832,278742331,NaN,"Outubro Rosa, consulte condições e agende já!",2023-10-17 17:15:09,NaN,NaN,NaN,NaN,True,True,False,False,False,False,True,True,False
121802,solangecruzdossantos@gmail.com,5338832,278742331,NaN,"Outubro Rosa, consulte condições e agende já!",2023-10-17 17:15:02,NaN,NaN,NaN,NaN,True,True,False,False,False,False,True,False,False
121803,mouragoreth5@gmail.com,5338832,278742331,NaN,"Outubro Rosa, consulte condições e agende já!",2023-10-17 17:15:24,NaN,NaN,NaN,NaN,True,True,False,False,False,False,True,False,False
121804,irani.santana.7568@gmail.com,5338832,278742331,NaN,"Outubro Rosa, consulte condições e agende já!",2023-10-17 17:15:09,NaN,NaN,NaN,NaN,True,True,False,False,False,False,True,False,False


In [26]:
df_orc = df_orc[df_orc['Motivo da rejeição'].isnull()]

# Drop rows where 'Relatório de spam' column is true
df_orc = df_orc[df_orc['Relatório de spam'] != True]

# Keep only the specified columns: 'Destinatário', 'Assunto', 'Aberto', 'Clicado'
columns_to_keep = ['Destinatário', 'Assunto', 'Aberto', 'Clicado']
df_orc = df_orc[columns_to_keep]

df_orc

,Destinatário,Assunto,Aberto,Clicado
0,ruizyara123@gmail.com,"Outubro Rosa, consulte condições e agende já!",False,False
1,edileneaparecidadelima97@gmail.com,"Outubro Rosa, consulte condições e agende já!",False,False
2,mariainestedesco2@gmail.com,"Outubro Rosa, consulte condições e agende já!",False,False
3,raizaamorim6@gmail.com,"Outubro Rosa, consulte condições e agende já!",False,False
4,marina_rolim@yahoo.com,"Outubro Rosa, consulte condições e agende já!",False,False
...,...,...,...,...
121801,cuidadoris75@gmail.com,"Outubro Rosa, consulte condições e agende já!",True,True
121802,solangecruzdossantos@gmail.com,"Outubro Rosa, consulte condições e agende já!",True,False
121803,mouragoreth5@gmail.com,"Outubro Rosa, consulte condições e agende já!",True,False
121804,irani.santana.7568@gmail.com,"Outubro Rosa, consulte condições e agende já!",True,False


In [33]:
# Get the counts of True and False values in 'Aberto' and 'Clicado' columns
count_aberto = df_orc['Aberto'].value_counts()
count_clicado = df_orc['Clicado'].value_counts()

# Calculate the proportion of True values in 'Aberto' and 'Clicado' columns
proportion_aberto = count_aberto.get(True, 0) / len(df_orc)
proportion_clicado = count_clicado.get(True, 0) / len(df_orc)

# Display the counts and proportions
print(f"Counts of 'Aberto':\n{count_aberto}")
print(f"\nCounts of 'Clicado':\n{count_clicado}")

print(f"\nProportion of True values in 'Aberto': {proportion_aberto:.2%}")
print(f"Proportion of True values in 'Clicado': {proportion_clicado:.2%}")

Counts of 'Aberto':
False    102444
True      19081
Name: Aberto, dtype: int64

Counts of 'Clicado':
False    120934
True        591
Name: Clicado, dtype: int64

Proportion of True values in 'Aberto': 15.70%
Proportion of True values in 'Clicado': 0.49%


In [34]:
# Get the counts of True and False values in 'Aberto' and 'Clicado' columns
count_aberto = df_orc['Aberto'].value_counts()
count_clicado = df_orc['Clicado'].value_counts()

# Calculate the proportion of True values in 'Aberto' and 'Clicado' columns
proportion_aberto = count_aberto.get(True, 0) / len(df_orc)
proportion_clicado_given_aberto_true = count_clicado.get(True, 0) / count_aberto.get(True, 1)

# Display the counts and proportions
print(f"Counts of 'Aberto':\n{count_aberto}")
print(f"\nCounts of 'Clicado':\n{count_clicado}")

print(f"\nProportion of True values in 'Aberto': {proportion_aberto:.2%}")
print(f"Proportion of True values in 'Clicado' given 'Aberto' is True: {proportion_clicado_given_aberto_true:.2%}")

Counts of 'Aberto':
False    102444
True      19081
Name: Aberto, dtype: int64

Counts of 'Clicado':
False    120934
True        591
Name: Clicado, dtype: int64

Proportion of True values in 'Aberto': 15.70%
Proportion of True values in 'Clicado' given 'Aberto' is True: 3.10%


In [27]:
df_orp.columns

Index(['Destinatário', 'ID do hub', 'ID da campanha de e-mail', 'Subtipo',
       'Assunto', 'Enviado às (seu fuso horário)', 'Motivo do não envio',
       'Mensagem não enviada', 'Motivo da rejeição', 'Mensagem rejeitada',
       'Enviados', 'Entregue', 'Suprimido', 'Recusado', 'Rejeição',
       'Relatório de spam', 'Aberto', 'Clicado', 'Assinatura cancelada'],
      dtype='object')

In [28]:
df_nac.columns

Index(['Destinatário', 'ID do hub', 'ID da campanha de e-mail', 'Subtipo',
       'Assunto', 'Enviado às (seu fuso horário)', 'Motivo do não envio',
       'Mensagem não enviada', 'Motivo da rejeição', 'Mensagem rejeitada',
       'Enviados', 'Entregue', 'Suprimido', 'Recusado', 'Rejeição',
       'Relatório de spam', 'Aberto', 'Clicado', 'Assinatura cancelada'],
      dtype='object')

In [29]:
df_nap.columns

Index(['Destinatário', 'ID do hub', 'ID da campanha de e-mail', 'Subtipo',
       'Assunto', 'Enviado às (seu fuso horário)', 'Motivo do não envio',
       'Mensagem não enviada', 'Motivo da rejeição', 'Mensagem rejeitada',
       'Enviados', 'Entregue', 'Suprimido', 'Recusado', 'Rejeição',
       'Relatório de spam', 'Aberto', 'Clicado', 'Assinatura cancelada'],
      dtype='object')

In [30]:
df_tlm.columns

Index(['Destinatário', 'ID do hub', 'ID da campanha de e-mail', 'Subtipo',
       'Assunto', 'Enviado às (seu fuso horário)', 'Motivo do não envio',
       'Mensagem não enviada', 'Motivo da rejeição', 'Mensagem rejeitada',
       'Enviados', 'Entregue', 'Suprimido', 'Recusado', 'Rejeição',
       'Relatório de spam', 'Aberto', 'Clicado', 'Assinatura cancelada'],
      dtype='object')